## POS tagging using modified Viterbi

### Data Preparation

In [139]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
import re
import nltk.data
import codecs
import os
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter


In [175]:
# Common Functions

#Function to get test sentences

def get_test_words(test_set, num_sent=5):
    
    random.seed(320)

    # choose random 5 sents
    rndom = [random.randint(1,len(test_set)) for x in range(num_sent)]
    
    # list of sents
    test_run = [test_set[i] for i in rndom]
    
    # list of tagged words
    test_run_base = [tup for sent in test_run for tup in sent]
    
    # list of untagged words
    test_tagged_words = [tup[0] for sent in test_run for tup in sent]
    
    return (test_tagged_words, test_run_base)

# function to get POS tag based on rules for a given word

def getPOSForWord(word):
    
    # rule 1. If the word contains *, output X
#    result = re.match("*\**", word)
#    if result:
#        return 'X'    
    # rule 2. If the word contains all digits output NUM
    result = re.match("^[0-9]+$", word)
    if result:
        return 'NUM'
    
    # rule 3. If the word contains digit,digit output NUM
    result = re.match("^[0-9]+,?[0-9]+$", word)
    if result:
        return 'NUM'
    
    # rule 4. If the word contains - eg. year-long, output ADJ
    result = re.match("^[a-z]+-[a-z]+$", word)
    if result:
        return 'ADJ'
    
    # rule 5. If the word begins with a capital letter, output NOUN
    result = re.match("^[A-Z]", word)
    if result:
        return 'NOUN'

    # rule 6. If the word contains digit.digit output NUM
    result = re.match("^[0-9]+.?[0-9]+$", word)
    if result:
        return 'NUM'
    
    # rule 7. If the word ends with ing, output VERB
    result = re.match("\w*ing", word)
    if result:
        return 'VERB'
    
    # rule 8. If the word ends with ly, output ADJ
    result = re.match("\w*ly", word)
    if result:
        return 'ADJ'
    
    # Catchall for no match
    return 'ADJ'

In [141]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [142]:
# Splitting into train and test
random.seed(320)
train_set, test_set = train_test_split(nltk_data, test_size=0.05)

print(len(train_set))
print(len(test_set))

3718
196


In [143]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95662

In [144]:
# Get the tokens 
tokens = [pair[0] for pair in train_tagged_words]

In [145]:
# Get the vocabulary
V = set(tokens)
print(len(V))

12051


In [146]:
# Get the tags
T = set([pair[1] for pair in train_tagged_words])
len(T)
T

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

### Build the vanilla Viterbi based POS tagger

In [147]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [148]:
# Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [149]:
# Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [150]:
# Tags Transition Matrix 
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [151]:
# Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

### Solve the problem of unknown words

Research paper from which techniques of unknown words have been used:
http://stp.lingfil.uu.se/~nivre/statmet/haulrich.pdf


### Approach 1.  using the most common Tag as the default tag for unknown words.

In [152]:
# Get the most frequent occuring tag.

data = Counter([pair[1] for pair in train_tagged_words])
data.most_common(1)[0][0]

'NOUN'

In [153]:
# Modifying the Viterbi Algorithm for unknown word
# Defaults to Adjective for unknown word
def Viterbi_Default_Tag(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        if pmax != 0:
            state_max = T[p.index(pmax)]
            state.append(state_max)
        else:
            state.append('NOUN')

    return list(zip(words, state))

### Approach 2.  Using a rule based approach for unknown words.

In [154]:
# Modifying the Viterbi Algorithm for unknown word
# Uses a rule based approach for unknown words
def Viterbi_Tag_Unknown_Words_RuleBased(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        if pmax != 0:
            state_max = T[p.index(pmax)]
            state.append(state_max)
        else:
            state.append(getPOSForWord(word))

    return list(zip(words, state))

#### Evaluating tagging accuracy

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [155]:
#Accuracy of the Vanilla Algorithm
test_tagged_words, test_run_base = get_test_words(test_set, 10)
tagged_seq = Viterbi(test_tagged_words)
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9543859649122807

In [156]:
#Accuracy of the Algorithm that handles unknown words with a default "NOUN" tag
test_tagged_words, test_run_base = get_test_words(test_set, 10)
tagged_seq = Viterbi_Default_Tag(test_tagged_words)
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9543859649122807

The Accuracy of this approach is same as the vanilla algorithm because the vanilla algorithm also chooses NOUN as default POS when the state probability is 0.

In [176]:
#Accuracy of the Algorithm that handles unknown words with a set of rules defined in the function getPOSForWord
test_tagged_words, test_run_base = get_test_words(test_set, 10)
tagged_seq = Viterbi_Tag_Unknown_Words_RuleBased(test_tagged_words)
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy


0.9578947368421052

The Rule Based POS Tagger is significantly better than the first two approaches.

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [158]:
# Read the sentences from the file.
doc = codecs.open('Test_sentences.txt', 'r', 'utf-8')
content = doc.read()

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

test_sentences = ' '.join(tokenizer.tokenize(content))

words = word_tokenize(test_sentences)


In [159]:
# Tags by the original POS tagger.
tagged_seq_vanilla = Viterbi(words)
print(tagged_seq_vanilla)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

In [160]:
# Tags by the Approach 1 of the POS tagger for handling unknown words.
tagged_seq_default_tag = Viterbi_Default_Tag(words)
print(tagged_seq_default_tag)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

In [161]:
# Tags by the Approach 2 of the POS tagger for handling unknown words.
# This uses a rule based tagger
tagged_seq_rule_based = Viterbi_Tag_Unknown_Words_RuleBased(words)
print(tagged_seq_rule_based)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

In [171]:
POS_comparison = [(j,i) for i, j in zip(tagged_seq_rule_based, tagged_seq_default_tag ) if i != j]

In [172]:
# Words tagged incorrectly by Vanilla Viterbi algorithm, but tagged correctly by modified algorithm
# For the given corpus of sentences.
POS_comparison

[(('2011', 'NOUN'), ('2011', 'NUM')),
 (('2013', 'NOUN'), ('2013', 'NUM')),
 (('2015', 'NOUN'), ('2015', 'NUM')),
 (('domineering', 'NOUN'), ('domineering', 'VERB')),
 (('2018', 'NOUN'), ('2018', 'NUM')),
 (('arriving', 'NOUN'), ('arriving', 'VERB'))]